# Approximate the integral $I=∫_{0}^{∞}\frac{1}{1+25x^2}dx$ such that the error of approximation is less than $10^{-10}$

Notice that $I$ can be computed by setting $u=5x$ then $du=5dx$ so
$$I=\frac{1}{5}\int_{0}^{\infty}\frac{1}{1+u^2}du=\left.\frac{1}{5} tan^{-1}(u)\right|_{0}^{\infty}=\frac{\pi}{10}$$

If we want to approximate this integration without calculate, we can setting $x=\frac{t}{1-t}$ then the range $[0,\infty)$ will transform into $[0,1]$ and we use Gauss–Legendre quadrature.

In [1]:
import argparse
import numpy as np
import mpmath as mp

In [2]:
def g(t):
    """Integrand on [0,1] after mapping x=t/(1-t)."""
    return 1.0 / (((1.0 - t) ** 2) + 25.0 * (t ** 2))


In [6]:
def gauss_legendre_adaptive(tol=1e-10, n0=32, nmax=4096, dps=80, verbose=True):
    mp.mp.dps = dps

    n = int(n0)
    if n < 8:
        n = 8


    xk, wk = np.polynomial.legendre.leggauss(n)
    t = 0.5 * (xk + 1.0)
    w = 0.5 * wk
    I_n = float(np.sum(g(t) * w))

    while True:
        if n * 2 > nmax:
            if verbose:
                print(f"[warn] Reached nmax={nmax} without meeting tol={tol}. Using current best.")
            return I_n, float("nan"), n

        n2 = n * 2
        xk2, wk2 = np.polynomial.legendre.leggauss(n2)
        t2 = 0.5 * (xk2 + 1.0)
        w2 = 0.5 * wk2
        I_n2 = float(np.sum(g(t2) * w2))
        est_err = abs(I_n2 - I_n)

        if verbose:
            print(
                "n={:5d} -> {:5d}  I_n={:.15e}  I_2n={:.15e}  |delta|={:.3e}".format(
                    n, n2, I_n, I_n2, est_err
                )
            )

        if est_err < tol / 2.0:
            return I_n2, est_err, n2

        n, I_n = n2, I_n2

In [7]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Integrate 1/(1+25x^2) on [0, inf) to target tol using Gauss–Legendre after mapping."
    )
    parser.add_argument("--tol", type=float, default=1e-10, help="target absolute accuracy (default: 1e-10)")
    parser.add_argument("--n0", type=int, default=32, help="initial number of nodes (default: 32)")
    parser.add_argument("--nmax", type=int, default=4096, help="maximum number of nodes (default: 4096)")
    parser.add_argument("--dps", type=int, default=80, help="mpmath decimal precision (default: 80)")
    parser.add_argument("--quiet", action="store_true", help="suppress per-iteration prints")
    args, _ = parser.parse_known_args()

    I, est_err, n = gauss_legendre_adaptive(
        tol=args.tol, n0=args.n0, nmax=args.nmax, dps=args.dps, verbose=(not args.quiet)
    )

    true = float(mp.pi / 10)
    true_err = abs(I - true)

    print("\nFinal result:       ", f"{I:.18f}")
    print("Nodes used:         ", n)
    print("Est. error (delta): ", f"{est_err:.3e}")
    print("Reference (pi/10):  ", f"{true:.18f}")
    print("True abs. error:    ", f"{true_err:.3e}")
    ok = (true_err <= args.tol) or (not np.isnan(est_err) and est_err <= args.tol)
    print("Meets tolerance?     ", ok)

n=   32 ->    64  I_n=3.141592653589793e-01  I_2n=3.141592653589794e-01  |delta|=1.110e-16

Final result:        0.314159265358979423
Nodes used:          64
Est. error (delta):  1.110e-16
Reference (pi/10):   0.314159265358979312
True abs. error:     1.110e-16
Meets tolerance?      True


# Approximate the integral $I=∫_{0}^{1}\frac{\ln(x)}{1+25x^2}dx$ such that the error of approximation is less than $10^{-10}$

## 1) Substitution \(x=e^{-t}\)

Let $x=e^{-t}\Rightarrow dx=-e^{-t}dt,\ \ln x=-t$. Then
$$
I=\int_{0}^{1}\frac{\ln x}{1+25x^2}\,dx
=\int_{0}^{\infty}\frac{-t\,e^{-t}}{1+25e^{-2t}}\,dt
=:\int_0^\infty h(t)\,dt,
\quad
h(t)=\frac{-t\,e^{-t}}{1+25e^{-2t}}.
$$
Now \(h(t)\) decays exponentially as $t \to \infty$, which is numerically stable.

## 2) Tail Error (Analytic Upper Bound)

Since $|h(t)|\le t e^{-t}$, for any $T>0$ we have
$$
\left|\int_T^\infty h(t)\,dt\right|
\le\int_T^\infty t e^{-t}\,dt
=(T+1)e^{-T}.
$$
Choose $T$ so that $(T+1)e^{-T}\le 5\times 10^{-11}$ to allocate half of the error budget to the tail.
For example, with $T=30$ we have $(31)e^{-30}\approx 2.9\times10^{-12}\ll 10^{-10}$.

## 3) Main Interval $[0,T]$and A-Posteriori Error Estimate

On $[0,T]$, use Gauss–Legendre quadrature. Double the number of nodes $n\to 2n$ and use
$$
E_{\text{main}}\approx |I_{2n}-I_n|
$$
as an a-posteriori error estimate. Iterate until $E_{\text{main}}\le 5\times10^{-11}$.

## 4) Total Error Guarantee

By the triangle inequality,
$$
|I_{\text{num}}-I|
\le E_{\text{main}}+E_{\text{tail}}
\le 5\times10^{-11}+5\times10^{-11}
=10^{-10}.
$$
Thus we obtain a rigorous $10^{-10}$ error bound without numerically integrating the tail.


In [14]:
import numpy as np
import mpmath as mp

In [22]:
def h(t):
    return (-t) * mp.e**(-t) / (1.0 + 25.0 * mp.e**(-2.0*t))

def tail_bound(T):
    return (T + 1.0) * mp.e**(-T)


In [23]:
def integrate_main_interval(T, tol_main=5e-11, n0=32, nmax=4096, verbose=True):

    n = max(8, int(n0))

    def quad_GL(n):
        # Gauss-Legendre nodes/weights on [-1,1]
        xk, wk = np.polynomial.legendre.leggauss(n)
        # map to [0,1]: u = (x+1)/2
        u = 0.5 * (xk + 1.0)
        w01 = 0.5 * wk
        # map [0,1] -> [0,T]: t = u*T, dt = T du
        t_vals = (u * T).astype(float)
        w = (w01 * T).astype(float)
        vals = np.array([h(tv) for tv in t_vals], dtype=float)
        return float(np.sum(vals * w))

    I_n = quad_GL(n)
    while True:
        if n * 2 > nmax:
            if verbose:
                print(f"[warn] reached nmax={nmax}, stopping with current estimate.")
            return I_n, n
        n2 = n * 2
        I_n2 = quad_GL(n2)
        err_est = abs(I_n2 - I_n)
        if verbose:
            print(f"n={n:5d} -> {n2:5d}  I_n={I_n:.15e}  I_2n={I_n2:.15e}  |Δ|={err_est:.3e}")
        if err_est < tol_main:
            return I_n2, n2
        n, I_n = n2, I_n2


In [24]:
def integrate_main_interval(T, tol_main=5e-11, n0=32, nmax=4096, verbose=True):

    n = max(8, int(n0))

    def quad_GL(n):

        xk, wk = np.polynomial.legendre.leggauss(n)

        u = 0.5 * (xk + 1.0)
        w01 = 0.5 * wk
        t_vals = (u * T).astype(float)
        w = (w01 * T).astype(float)
        vals = np.array([h(tv) for tv in t_vals], dtype=float)
        return float(np.sum(vals * w))

    I_n = quad_GL(n)
    while True:
        if n * 2 > nmax:
            if verbose:
                print(f"[warn] reached nmax={nmax}, stopping with current estimate.")
            return I_n, n
        n2 = n * 2
        I_n2 = quad_GL(n2)
        err_est = abs(I_n2 - I_n)
        if verbose:
            print(f"n={n:5d} -> {n2:5d}  I_n={I_n:.15e}  I_2n={I_n2:.15e}  |Δ|={err_est:.3e}")
        if err_est < tol_main:
            return I_n2, n2
        n, I_n = n2, I_n2

In [25]:
def integrate_substitution_only(target=1e-10, T=30.0, n0=32, nmax=4096, dps=80, verbose=True):

    mp.mp.dps = dps

    E_tail = tail_bound(T)
    if verbose:
        print(f"T = {T},   tail_bound(T) = {float(E_tail):.3e}")
    if E_tail > target / 2.0:
        raise ValueError(
            f"T too small: tail_bound(T) = {float(E_tail):.3e} > {target/2:.1e}. "
            f"Increase T (e.g., T=27 or 30)."
        )


    I_main, n_used = integrate_main_interval(
        T, tol_main=target/2.0, n0=n0, nmax=nmax, verbose=verbose
    )


    if verbose:
        print("\nResult (main only) =", f"{I_main:.12f}")
        print(f"Guaranteed total error ≤ {target:.1e}  "
              f"(main ≤ {target/2:.1e}, tail ≤ {target/2:.1e})")
    return I_main, n_used, float(E_tail)

In [26]:
if __name__ == "__main__":
    I, n_used, E_tail = integrate_substitution_only(
        target=1e-10, T=30.0, n0=32, nmax=4096, dps=80, verbose=True
    )
    print("\nI ≈", f"{I:.12f}")
    print("nodes used:", n_used, "  tail bound:", f"{E_tail:.3e}")

T = 30.0,   tail_bound(T) = 2.901e-12
n=   32 ->    64  I_n=-5.454428922834451e-01  I_2n=-5.454445634186502e-01  |Δ|=1.671e-06
n=   64 ->   128  I_n=-5.454445634186502e-01  I_2n=-5.454445634169020e-01  |Δ|=1.748e-12

Result (main only) = -0.545444563417
Guaranteed total error ≤ 1.0e-10  (main ≤ 5.0e-11, tail ≤ 5.0e-11)

I ≈ -0.545444563417
nodes used: 128   tail bound: 2.901e-12
